# Binance用オーダー発行API

`BinanceMarket`は、通過ペアー毎に以下の機能を提供します。

* 蓄積用データベースの生成（Sqlite)
* 約定データのダウンロードと更新
  * 過去データのダウンロード(Binance Webサイト)
  * リアルタイム更新(WebSocket)
* 約定データのDBからの取得と足の作成
  * 任意の期間の足の作成(`ohlcv`メソッド)
  * VAP(Value At Price)の計算(`vap`メソッド)
* オーダーの発行・キャンセル（REST API)　KEYとSECRETの設定が必要です。

また通過ペアーの設定は`BinanceConfig`クラスに定義してあります。パラーメータを修正することで任意の通過ペアーに対応可能ですが、テスト完了して提供しているものは以下になります。
* `BinanceConfig.BTCUSDT` 本番用BTCUSDTペアー（現物）
* `BinanceConfig.TEST_BTCUSDT`　テストネット用BTCUSDTペアー（現物）

## 本サンプルの機能

本サンプルでは以下を行います。

* オーダーの発行(Limit, Market)
* オーダーの検索
* オーダーのキャンセル
* 資産残高の確認

### 利用している関数
* オーダー発行
  * `limit_order(side, price, size)` リミットオーダーの発行
  * `market_order(side, size)`　マーケットオーダーの発行
* オーダー状況確認
  * `order_status`　オーダーステータスの確認（約定済みも含めた直近のもの）
  * `open_orders`　未約定一覧
* オーダーキャンセル
  * `cancel_order(order_id)`　指定したIDのオーダキャンセル
  * `cancel_all_orders()`　未約定のオーダーのキャンセル
* アカウント情報
  * `account`　アカウント情報（残高など）

## API KEYについて（注：まずはテストネットで試してください）
動作させるためにはAPI KEYが必要です。
ローカル環境では、以下の値を環境変数に指定してください。
* BINANCE_API_KEY   APIキー
* BINANCE_API_SERECT  シークレット

Google Clabでは、userdataに設定してください（Colabのページの左側ツールバーにある🔑マークをクリック）。

## 必要ライブラリのインストール

`rbot`がBotFrameWorkの本体です。必要なライブラリを合わせてインストールします。２回目は実行不要です。

In [ ]:
# 必要ライブラリのインストール
! pip install --upgrade pip

# rbotがbot frameworkになります。PyPiに登録されているので、pipでインストールできます。
# まだ開発初期段階なので、インターフェースが変わる可能性があります。そのためバージョン指定しています。
#! pip install --upgrade rbot==0.2.4


In [ ]:
# 必要ライブラリのインストール。環境によっては要・不要があるので適宜修正してください。
! pip install json2html

## Python コード本体

### 必要ライブラリのインポート

In [ ]:
import rbot
# BinanceMarketクラスは、BinanceのAPIを利用して、取引所の情報を取得するクラス
from rbot import BinanceMarket

# BinanceConfigクラスは、BinanceMarketクラスのコンストラクタに渡す設定クラス
from rbot import BinanceConfig

from rbot import init_log, init_debug_log
#init_debug_log()

print("rbot version =", rbot.__version__)

In [ ]:
# 必要ライブラリーのインポート
from json2html import *
from IPython.display import HTML

### BinanceMarketオブジェクトの生成

引数にBinanceConfigを設定してオブジェクトを生成します。

In [ ]:
# BinanceMarketクラスのインスタンスを生成(テストネットのBTCUSDT現物取引の設定)
config = BinanceConfig.TEST_BTCUSDT

# Google Colabの場合は、userdataからAPIキーを取得
# 通常は、環境変数に設定してください。
if 'google.colab' in str(get_ipython()):
    from google.colab import userdata
    # Google Colabの場合は、APIキーを入力してください。
    config.api_key = userdata.get("BINANCE_API_KEY")
    config.api_secret = userdata.get("BINANCE_API_SECRET")

# json2htmlを利用して、設定情報を表示
HTML(json2html.convert(config.__str__()))

In [ ]:
# BinanceMarketクラスのインスタンスを生成(BTCUSDT現物取引の設定)
market = BinanceMarket(config)

# BinanceMarketのインスタンスを表示すると格納されているデータの情報が表示されます。
market

In [ ]:
from time import sleep

# WebSocketを利用して、取引所の情報を取得する
market.start_market_stream()

sleep(1) # データが揃うまで少し待つ

In [ ]:
# 一旦　過去のオーダーを全部キャンセルする
market.cancel_all_orders()

## オーダー発行テスト

### Limit order

In [ ]:
# 現在のアカウント情報を取り出す
account = market.account
account

In [ ]:
# accountにはすべての通貨の残高が入っているので、BTCとUSDTの残高を表示する
print("BTC 残高 =", account['BTC'])
print("USDT残高 =", account['USDT'])

In [ ]:
# 現在の板情報を取り出す
bids, asks, = market.board

bid_edge = bids['price'][0]
ask_edge = asks['price'][0]

In [ ]:
# 板より１００ドル安いところへ0.0001BTCの買い注文を出す
buy_limit_order = market.limit_order("BUY", bid_edge - 100, 0.001)
buy_limit_order

In [ ]:
# 未約定のオーダーを表示する。オーダーが１つ増えているはず。
market.open_orders

In [ ]:
# 板より１００ドル高いところへ0.0001BTCの売り注文を出す
sell_limit_order = market.limit_order("SELL", ask_edge + 100, 0.001)
sell_limit_order

In [ ]:
# 未約定のオーダーを表示する。オーダーが１つ増えているはず（合計２つ）
market.open_orders

In [ ]:
# 最新の残高を確認。注文中の資産がLockされているはず。
account = market.account
print("BTC 残高 =", account['BTC'])
print("USDT残高 =", account['USDT'])

In [ ]:
# 売りオーダーをキャンセルする

id_to_cancel = buy_limit_order[0].order_id

print("キャンセルするオーダーID =", id_to_cancel)

market.cancel_order(sell_limit_order[0].order_id)

In [ ]:
# キャンセルしたオーダーがなくなっていることを確認する
market.open_orders

In [ ]:
# 最新の残高を確認。キャンセルされた分の資産のLockが解放される。
account = market.account
print("BTC 残高 =", account['BTC'])
print("USDT残高 =", account['USDT'])

## Market order

In [ ]:
# market orderを出す

market_order = market.market_order("BUY", 0.001)   # 0.001BTCを市場価格で買う
market_order

In [ ]:
# 直近のオーダーを表示する。status "Filled"のオーダーが１つ追加されているはず。
# なお大きなオーダーを出すと複数に分割される場合がある。

market.order_status

In [ ]:
# 最新の残高を確認。約定した分だけBTCが増えて、USDTの残高が減っているはず。
account = market.account
print("BTC 残高 =", account['BTC'])
print("USDT残高 =", account['USDT'])